## 한국 음식 분류 모델

### 데이터 준비

In [1]:
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from PIL import Image  # 이미지 파일을 연결하고 변환가능한 객체 생성해주는 라이브러리
import os, glob
import numpy as np

In [2]:
# 분류데이터 로딩을 위한 카테고리 지정
# 데이터 저장 기본 폴더

# 치킨 이미지는 ./Kfood/Chicken/*.jpg
root_dir = "./kfood/"

# 카테고리 명령 지정 (1)
categories = ["Chicken", "Dolsotbab", "Jeyugbokkeum", "Kimchi", 
                "Samgyeobsal", "SoybeanPasteStew"]
nb_classes = len(categories)

In [3]:
# 이미지 크기 지정 (2)
image_width = 64
image_height = 64

In [4]:
# 데이터 변수
X = []  # 이미지 데이터
Y = []  # 레이블 데이터

for idx, category in enumerate(categories):
    image_dir = root_dir + category
    files = glob.glob(image_dir + "/" + "*.jpg")
    print(image_dir + "/" +"*.jpg")

    for i, f in enumerate(files):
        # 이미지 로딩 (3)
        # 이미지 전처리 코드
        img = Image.open(f)
        img = img.convert('RGB')
        img = img.resize((image_width, image_height))  # 64x64
        data = np.asarray(img)  # 이미지데이터 array로 변경
        X.append(data)
        Y.append(idx)

./kfood/Chicken/*.jpg
./kfood/Dolsotbab/*.jpg
./kfood/Jeyugbokkeum/*.jpg
./kfood/Kimchi/*.jpg
./kfood/Samgyeobsal/*.jpg
./kfood/SoybeanPasteStew/*.jpg


In [5]:
# 이미지 형태 변경 (전체 data array로 변경)
X = np.array(X)
Y = np.array(Y)

In [6]:
# 학습 데이터와 테스트 데이터 나누기 (4)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)
xy = (X_train, X_test, Y_train, Y_test)

# 데이터 파일 저장 (5)
# 전처리 완료된 data파일 저장 - npy 파일로 저장 (np array 파일로 저장)
np.save(root_dir + 'kfood.npy', xy)

# 대용량 이미지 처리하거나 자연어 처리를 진행할 경우 중간파일을 저장할 필요가 있음
# 자연어 처리 같은 경우는 단어 사전을 저장했다가 서비스 등에 이용해야 함
# 보통 numpy의 array 형식을 가장 많이 사용함
# 딕셔너리 저장도 npy 형식으로 저장하면 훨씬 효울적

/home/ubuntu/anaconda3/envs/tensorflow2_p38/lib/python3.8/site-packages/numpy/core/_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


### 모델 생성

In [7]:
## 사용할 모델 라이브러리 import
import sys, os
from keras.models import Sequential  # 모델 구조를 생성하는 라이브러리
from keras.layers import Convolution2D  # 합성곱 층을 생성하는 라이브러리(2차원 처리)
from keras.layers import MaxPooling2D  # 합성곱 층에서 생성된 특성맵을 단순화하는 층(2차원 처리)
from keras.layers import Activation  # 활성화 함수 지정
from keras.layers import Dropout  # 규제적용 층
from keras.layers import Flatten  # 데이터를 1차원으로 변환시켜주는 층
from keras.layers import Dense  # 밀집층
from keras.utils import np_utils  
import numpy as np

In [8]:
# 초기 설정
root_dir = "./kfood/"
categories = ["Chicken", "Dolsotbab", "Jeyugbokk-eum", "Kimchi", 
                "Samgyeobsal", "SoybeanPasteStew"]
nb_classes = len(categories)
image_size = 64

In [9]:
# 저장해놓은 npy 파일 로드 후 최종 전처리 함수
def load_dataset():
    x_train, x_test, y_train, y_test = np.load("./kfood/kfood.npy",allow_pickle=True)
    
    # 스케일링코드
    x_train = x_train.astype('float') / 255
    x_test = x_test.astype('float') / 255
    
    # 폴더별 카테고리화(label 생성)
    y_train = np_utils.to_categorical(y_train, nb_classes)
    y_test = np_utils.to_categorical(y_test, nb_classes)
    
    return  x_train, x_test, y_train, y_test

In [10]:
x, x1, y, y1 = load_dataset()

In [11]:
type(y[0][0])

numpy.float32

In [12]:
# 입력 이미지 크기에 따른 모델 구성 함수
# 모델은 기본 cnn 모델이며 dropout으로 규제 설정 되어 있음
def build_model(in_shape):
    model = Sequential()
    model.add(Convolution2D(32, 3, 3, padding='Same', 
                input_shape=in_shape))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
#     model.add(Dropout(0.25))  # 25% 필터 끄기
    model.add(Convolution2D(64, 3, 3, padding='same'))
    model.add(Activation('relu'))
    model.add(Convolution2D(64, 3, 3))
    model.add(MaxPooling2D(pool_size=(2,2),padding='same'))
#     model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.3))
    model.add(Dense(nb_classes))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', 
                    optimizer='rmsprop', 
                    metrics=['accuracy'])

    return model

In [13]:
# 모델 생성후 학습된 모델 반환하는 함수
def model_train(x, y):
    model = build_model(x.shape[1:])
    model.fit(x, y, batch_size=32, epochs=30)  # batch_size=32 기본값
    return model

In [14]:
# 모델 평가함수
def model_eval(model, x, y):
    score = model.evaluate(x, y)
    print('loss=', score[0])
    print('accuracy=', score[1])

In [15]:
# 모델 학습 및 평가
x_train, x_test, y_train, y_test = load_dataset()
model = model_train(x_train, y_train)
model_eval(model, x_test, y_test)

Epoch 1/30
4/4 [==============================] - 3s 58ms/step - loss: 1.7385 - accuracy: 0.2661
Epoch 2/30
4/4 [==============================] - 0s 23ms/step - loss: 1.6322 - accuracy: 0.3486
Epoch 3/30
4/4 [==============================] - 0s 10ms/step - loss: 1.5700 - accuracy: 0.2661
Epoch 4/30
4/4 [==============================] - 0s 10ms/step - loss: 1.4577 - accuracy: 0.4037
Epoch 5/30
4/4 [==============================] - 0s 6ms/step - loss: 1.3508 - accuracy: 0.3394
Epoch 6/30
4/4 [==============================] - 0s 6ms/step - loss: 1.2348 - accuracy: 0.4037
Epoch 7/30
4/4 [==============================] - 0s 6ms/step - loss: 1.1560 - accuracy: 0.5046
Epoch 8/30
4/4 [==============================] - 0s 5ms/step - loss: 0.9823 - accuracy: 0.6055
Epoch 9/30
4/4 [==============================] - 0s 4ms/step - loss: 1.1370 - accuracy: 0.4404
Epoch 10/30
4/4 [==============================] - 0s 4ms/step - loss: 0.9338 - accuracy: 0.6055
Epoch 11/30
4/4 [==================

### 1차 모델
- 과소적합으로 보여짐
    - 모델에 dropout을 통해 과한 규제
        - dropout을 제거
    - (146, 64, 64, 2)의 학습데이터를 사용 - 딥러닝에 사용하기에 data가 너무 적다.
    - 위 두 문제가 아니면 층을 추가하고 필터 수를 늘림
    - 에포크 수를 증가시킬 수 있음

In [16]:
# 모델 저장
model.save("./kfood/kfood_model.h5")

### 모델 로드하여 확인하기

In [17]:
import sys, os
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.utils import np_utils
from keras.models import load_model
from PIL import Image
import numpy as np

In [18]:
# 테스트 이미지 목록 (1)
image_files = ["kfood/Chicken/chicken_01.jpg",
               "kfood/Chicken/chicken_02.jpg",
               "kfood/Kimchi/kimchi15.jpg",
               "kfood/Kimchi/kimchi07.jpg",
               "kfood/Samgyeobsal/Samgyeobsal04.jpg"]
image_size = 64
nb_classes = len(image_files)
categories = ["Chicken", "Dolsotbab", "Jeyugbokk-eum", "Kimchi",
              "Samgyeobsal", "SoybeanPasteStew"]

In [19]:
image_size = 64
X = []
files = []

# 테스트이미지 전처리
for fname in image_files:
    img = Image.open(fname)
    img = img.convert('RGB')
    img = img.resize((image_size, image_size))
    in_data = np.asarray(img)
    in_data = in_data.astype('float')/255
    X.append(in_data)  # list
    files.append(fname)

X = np.array(X)  # array

In [20]:
X

array([[[[0.56862745, 0.11372549, 0.05098039],
         [0.6627451 , 0.18039216, 0.09803922],
         [0.50980392, 0.14901961, 0.09019608],
         ...,
         [0.35294118, 0.30196078, 0.24313725],
         [0.29803922, 0.24705882, 0.19607843],
         [0.32156863, 0.24705882, 0.18431373]],

        [[0.66666667, 0.10588235, 0.        ],
         [0.58039216, 0.05490196, 0.        ],
         [0.42352941, 0.04313725, 0.01960784],
         ...,
         [0.30980392, 0.23529412, 0.18431373],
         [0.30980392, 0.25490196, 0.20784314],
         [0.3254902 , 0.2627451 , 0.20784314]],

        [[0.55686275, 0.22352941, 0.19215686],
         [0.38431373, 0.01568627, 0.00392157],
         [0.29411765, 0.01568627, 0.02352941],
         ...,
         [0.34509804, 0.29019608, 0.23529412],
         [0.30588235, 0.24313725, 0.19607843],
         [0.30980392, 0.25882353, 0.21568627]],

        ...,

        [[0.72156863, 0.61176471, 0.5372549 ],
         [0.68235294, 0.55294118, 0.48235294]

In [21]:
# 모델 파일 읽어오기
model = load_model('./kfood/kfood_model.h5')

In [22]:
# 예측 실행  (4)
pre = model.predict(X)  # 예측에 대한 확률

In [23]:
# 예측 결과 출력 (5)
for i, p in enumerate(pre):
    y = p.argmax()  # 예측확률이 가장 큰 값의 index 호출코드
    print("입력:", files[i])
    # 예측 결과 아래 예시와 같이 출력
    print("예측:", "[", y, "]", categories[y], "/ Score", p[y])
    
    # 실제로 그렇게 좋은 모델은 아니다.

입력: kfood/Chicken/chicken_01.jpg
예측: [ 3 ] Kimchi / Score 0.7255892
입력: kfood/Chicken/chicken_02.jpg
예측: [ 3 ] Kimchi / Score 0.7849758
입력: kfood/Kimchi/kimchi15.jpg
예측: [ 3 ] Kimchi / Score 0.71715957
입력: kfood/Kimchi/kimchi07.jpg
예측: [ 3 ] Kimchi / Score 0.96639687
입력: kfood/Samgyeobsal/Samgyeobsal04.jpg
예측: [ 4 ] Samgyeobsal / Score 0.6747752


### 위 테스트 결과는 학습이미지를 이용해서 테스트 한 결과
- 모델 성능보다 분류를 잘 할 수 있음